Processamento de dados:
- criar tabela com todos os MQLs, com todas as informações disponíveis para cada um.
- criar uma coluna com quais MQLs fecharam acordo e quais foram perdidos.

Perguntas a serem respondidas:
- Qual foi a taxa de conversão total?
- Qual foi a taxa de conversão de cada origem?
- Qual foi a taxa de conversão de cada página inicial?
- Para cada SR, qual a receita média declarada dos leads?
- Para cada SR, quantos de cada business_type a pessoa converteu?
- Para cada SR, quantos de cada lead_behaviour_profile a pessoa converteu?
- Para cada SR, quantos de cada lead_type a pessoa converteu?

Extra:

Caso consiga resolver as questões acima com facilidade, aproveite a base "irmã" dessa para fazer outras explorações que você ache interessante!

https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

Lembre de separar novamente em 2 etapas: 
1. Processamento dos dados, para criar uma tabela "analítica".
2. Análise exploratória dos dados.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_closed = pd.read_csv('../dados/olist_closed_deals_dataset.csv', sep=",")
df_closed['contract'] = "accepted"
dt_marketing = pd.read_csv('../dados/olist_marketing_qualified_leads_dataset.csv', sep=",")

In [3]:
df_total_mql = df_closed.merge(dt_marketing, left_on=['mql_id'], right_on=['mql_id'], how='right')
df_total_mql = df_total_mql.replace(np.nan, 0, regex=True)
df_total_mql['seller_id'].fillna(0, inplace=True)
df_total_mql.loc[(df_total_mql['seller_id'] == 0), 'contract'] = "lost"

In [8]:
### Taxa de conversão total - somar todas a receita mensal e dividir pela quantidade total de mql
df_lost = df_total_mql[(df_total_mql['contract'] != "lost")].count()
tx_total = (df_lost['mql_id'] / df_total_mql['mql_id'].count()) * 100

print("A taxa de conversão total de mql ativos e total é de: ", tx_total, "%")

A taxa de conversão total de mql ativos e total é de:  10.525 %


In [10]:
##Qual foi a taxa de conversão de cada origem?

df_origin = df_total_mql.groupby(['origin']).agg(origin_origin=('origin', 'count')).reset_index()
df_origin['tx_conversao'] = (df_origin['origin_origin'] / df_total_mql['origin'].count()) * 100

print("A taxa de conversão para cada origin é: \n", df_origin)



A taxa de conversão para cada origin é: 
                origin  origin_origin  tx_conversao
0                   0             60        0.7500
1      direct_traffic            499        6.2375
2             display            118        1.4750
3               email            493        6.1625
4      organic_search           2296       28.7000
5               other            150        1.8750
6   other_publicities             65        0.8125
7         paid_search           1586       19.8250
8            referral            284        3.5500
9              social           1350       16.8750
10            unknown           1099       13.7375


In [17]:
##Qual foi a taxa de conversão de cada página inicial?
df_page_id = df_total_mql.groupby(['landing_page_id']).agg(count_page_id=('landing_page_id', 'count')).reset_index()
df_page_id['tx_conversao'] = (df_page_id['count_page_id'] / df_total_mql['landing_page_id'].count()) * 100

print("A taxa de conversão para cada página é: \n", df_page_id)

A taxa de conversão para cada página é: 
                       landing_page_id  count_page_id  tx_conversao
0    007f9098284a86ee80ddeb25d53e0af8             66        0.8250
1    01850876d34b43494b5f9879c366877b              1        0.0125
2    0218f6be0b76aca72ab4d00ee9e8cf10             48        0.6000
3    03c7c0ace395d80182db07ae2c30f034              1        0.0125
4    04a75b75d5f4f3a16b0178fbe382da00              1        0.0125
..                                ...            ...           ...
490  fd7c4cdfbfb7964b441618d54f88c38b              1        0.0125
491  fd7d977e75d1ac5cfe6b93ffabb150b1              1        0.0125
492  fe82aeadeaea2186478fe6353d6d5301              8        0.1000
493  ff3416719616ad4921acb8c74e21d6f0             11        0.1375
494  ffc62d6945fd1423539ec6d96326c37c              1        0.0125

[495 rows x 3 columns]


In [23]:
##Para cada SR, qual a receita média declarada dos leads?
df_sr_leads = df_total_mql.groupby(['sr_id']).agg(count_sr_leads=('declared_monthly_revenue', 'count'),
                                    sum_sr_leads=('declared_monthly_revenue', 'sum'),
                                    mean_sr_leads=('declared_monthly_revenue', 'mean')).reset_index()

print("A receita média mensal do SR é: \n", df_sr_leads)

A receita média mensal do SR é: 
                                sr_id  count_sr_leads  sum_sr_leads  \
0                                  0            7158           0.0   
1   060c0a26f19f4d66b42e0d8796688490              32           0.0   
2   068066e24f0c643eb1d089c7dd20cd73              27      190000.0   
3   0a0fb2b07d841f84fb6714e35c723075               1        6000.0   
4   2695de1affa7750089c0455f8ce27021              59      100000.0   
5   34d40cdaf94010a1d05b0d6212f9e909              10      190000.0   
6   495d4e95a8cf8bbf8b432b612a2aa328              63      420000.0   
7   4b339f9567d060bcea4f5136b9f5949e               9      250000.0   
8   4ef15afb4b2723d8f3d81e51ec7afefe             133    50180000.0   
9   56bf83c4bb35763a51c2baab501b4c67              24      300000.0   
10  6565aa9ce3178a5caf6171827af3a9ba              74           0.0   
11  6aa3b86a83d784b05f0e37e26b20860d               1        8000.0   
12  85fc447d336637ba1df43e793199fbc8              64    

In [37]:
##Para cada SR, quantos de cada business_type a pessoa converteu?

##quantidade de cada business_type total
df_business_type = df_total_mql.groupby(['business_type']).agg(count_business_type=('business_type', 'count')).reset_index()

##quantidade de business_type para cada SR
df_sr_business_type = df_total_mql.groupby(['sr_id', 'business_type']).agg(count_sr_business=('business_type', 'count')).reset_index()

#merge entre as bases
df_sr_business_type = df_sr_business_type.merge(df_business_type, left_on=['business_type'], right_on=['business_type'], how='left')



df_sr_business_type['mean_sr_business'] = (df_sr_business_type['count_sr_business'] / df_sr_business_type['count_business_type']) * 100


df_sr_business_type

,sr_id,business_type,count_sr_business,count_business_type,mean_sr_business
0,0,0,7158,7168,99.860491
1,060c0a26f19f4d66b42e0d8796688490,manufacturer,9,242,3.719008
2,060c0a26f19f4d66b42e0d8796688490,reseller,23,587,3.918228
3,068066e24f0c643eb1d089c7dd20cd73,manufacturer,6,242,2.479339
4,068066e24f0c643eb1d089c7dd20cd73,reseller,21,587,3.577513
5,0a0fb2b07d841f84fb6714e35c723075,other,1,3,33.333333
6,2695de1affa7750089c0455f8ce27021,0,1,7168,0.013951
7,2695de1affa7750089c0455f8ce27021,manufacturer,16,242,6.611570
8,2695de1affa7750089c0455f8ce27021,reseller,42,587,7.155026
9,34d40cdaf94010a1d05b0d6212f9e909,manufacturer,1,242,0.413223


In [42]:
##Para cada SR, quantos de cada lead_behaviour_profile a pessoa converteu?

##quantidade de cada lead_behaviour_profile total
df_lead_behaviour_profile = df_total_mql.groupby(['lead_behaviour_profile']).agg(count_lead_behaviour_profile=('lead_behaviour_profile', 'count')).reset_index()

##quantidade de lead_behaviour_profile para cada SR
df_sr_lead_behaviour_profile = df_total_mql.groupby(['sr_id', 'lead_behaviour_profile']).agg(count_sr_behaviour_profile=('lead_behaviour_profile', 'count')).reset_index()

#merge entre as bases
df_sr_lead_behaviour_profile = df_sr_lead_behaviour_profile.merge(df_lead_behaviour_profile, left_on=['lead_behaviour_profile'], right_on=['lead_behaviour_profile'], how='left')



df_sr_lead_behaviour_profile['mean_sr_behaviour_profile'] = (df_sr_lead_behaviour_profile['count_sr_behaviour_profile'] / df_sr_lead_behaviour_profile['count_lead_behaviour_profile']) * 100


df_sr_lead_behaviour_profile

,sr_id,lead_behaviour_profile,count_sr_behaviour_profile,count_lead_behaviour_profile,mean_sr_behaviour_profile
0,0,0,7158,7335,97.586912
1,060c0a26f19f4d66b42e0d8796688490,0,8,7335,0.109066
2,060c0a26f19f4d66b42e0d8796688490,cat,15,407,3.685504
3,060c0a26f19f4d66b42e0d8796688490,eagle,2,123,1.626016
4,060c0a26f19f4d66b42e0d8796688490,shark,1,24,4.166667
...,...,...,...,...,...
89,fbf4aef3f6915dc0c3c97d6812522f6a,0,16,7335,0.218132
90,fbf4aef3f6915dc0c3c97d6812522f6a,cat,30,407,7.371007
91,fbf4aef3f6915dc0c3c97d6812522f6a,eagle,9,123,7.317073
92,fbf4aef3f6915dc0c3c97d6812522f6a,shark,2,24,8.333333


In [46]:
##Para cada SR, quantos de cada lead_type a pessoa converteu?

##quantidade de cada lead_type total
df_lead_type = df_total_mql.groupby(['lead_type']).agg(count_lead_type=('lead_type', 'count')).reset_index()

##quantidade de lead_type para cada SR
df_sr_lead_type = df_total_mql.groupby(['sr_id', 'lead_type']).agg(count_sr_lead_type=('lead_type', 'count')).reset_index()

#merge entre as bases
df_sr_lead_type = df_sr_lead_type.merge(df_lead_type, left_on=['lead_type'], right_on=['lead_type'], how='left')



df_sr_lead_type['mean_sr_lead_type'] = (df_sr_lead_type['count_sr_lead_type'] / df_sr_lead_type['count_lead_type']) * 100


df_sr_lead_type

,sr_id,lead_type,count_sr_lead_type,count_lead_type,mean_sr_lead_type
0,0,0,7158,7164,99.916248
1,060c0a26f19f4d66b42e0d8796688490,industry,8,123,6.504065
2,060c0a26f19f4d66b42e0d8796688490,offline,1,104,0.961538
3,060c0a26f19f4d66b42e0d8796688490,online_beginner,3,57,5.263158
4,060c0a26f19f4d66b42e0d8796688490,online_big,2,126,1.587302
...,...,...,...,...,...
118,fbf4aef3f6915dc0c3c97d6812522f6a,online_beginner,1,57,1.754386
119,fbf4aef3f6915dc0c3c97d6812522f6a,online_big,8,126,6.349206
120,fbf4aef3f6915dc0c3c97d6812522f6a,online_medium,30,332,9.036145
121,fbf4aef3f6915dc0c3c97d6812522f6a,online_small,6,77,7.792208
